In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [ ]:
import intervaltree
import itertools
import pint
from matplotlib import colormaps
import warnings
import json
import pathlib

In [ ]:
%autoreload 2
import njl_corf
from njl_corf.pyfcctab import ureg
import njl_corf.pyfcctab as pyfcctab
import njl_corf.pyoscar as pyoscar
import njl_corf.band_figure as band_figure
import njl_corf.wrc27_views as wrc27_views

In [ ]:
allocation_database = pyfcctab.read(skip_additionals=False)

In [ ]:
oscar_database = pyoscar.read()
oscar_database = pyoscar.merge_sensors(
    oscar_database,
    {
        "AMSR": "AMSR*",
        "ATMS": "ATMS*",
        "AWS": "AWS*",
        "MWTS": "MWTS*",
        "SSMIS": "SSMIS*",
        "AMSU": "AMSU*",
        #        "MWRI": "MWRI*",
        "MWS": "MWS*",
        "MTVZA": "MTVZA*",
        "MWI": "MWI*",
        "SRAL": "SRAL*",
        "OSCAT": "OSCAT*",
        "Altimetry": ["ALT*", "RA*", "Altimeter*"],
        "Precipitation": ["PR*", "DPR*", "Rainradar"],
        "Poseidon": "Poseidon*",
        "SWIM": "SWIM*",
        "MWR": "MWR*",
        "WSF": "WSF*",
        "MLS": "MLS*",
        "ICI": "ICI*",
    },
)

In [ ]:
# help(matplotlib.patches.Polygon)

In [ ]:
ai_ranges = wrc27_views.get_ai_info()

In [ ]:
collation_rules = {
    "Space research": ["Space research*"],
    "Mobile": ["Mobile", "Mobile*"],
    "Fixed": ["Fixed", "Fixed*"],
    "Aeronautical": ["Aeronautical*"],
    "EESS (passive)": ["Earth exploration-satellite (passive)"],
    "EESS (active)": ["Earth exploration-satellite (active)"],
    "EESS (comms)": ["Earth exploration-satellite (*to*)"],
    "Amateur": ["Amateur*"],
    "Radionavigation": ["Radionavigation*"],
}

In [ ]:
config = band_figure.FigureConfiguration.load()
ureg.setup_matplotlib()
fig, ax = plt.subplots(figsize=[4, 8])  # , layout="constrained")
band_figure.views_plot(
    frequency_range=ai_ranges["1.12c"],
    allocation_database=allocation_database,
    oscar_database=oscar_database,
    ax=ax,
    figure_configuration=config,
    collation_rules=collation_rules,
)
# plt.savefig("test.pdf")

In [ ]:
bands = allocation_database.itu[183 * ureg.GHz]
print([band.allocations[0].modifiers for band in bands])

In [ ]:
??allocation_database.itu.data.split_overlaps

In [ ]:
x = slice(1, 2)
help(x)

In [ ]:
import json

In [ ]:
json.dumps({"colors_scheme": SERVICE_COLORS})